# 1 Artificial Neura Network

## 1.4 Around the dataset of MNIST

### 1.4.1 Preparation

 Constructing the Neuro Network,

In [37]:
mutable struct NetInfo  # pay attention to the mutatle
# nodes of hierarchy of input，hidden，output
    inodes::Int # input hierarchy nodes
    hnodes::Int # hidden 
    onodes::Int # output
# matrix of linking weights
    wih # input to hidden
    who # hidden to output
#  Learning Rate
    lr::Float64
end

Activation Function，sigmoid is Here

In [38]:
sigmoid(x) = 1 / (1 + exp(-x))

sigmoid (generic function with 1 method)

Initiating the Neuro Network

In [39]:
using Distributions

In [40]:
function InitNet(inodes::Int, hnodes::Int, onodes::Int, lr::Float64)
    wih = rand(Normal(0, hnodes^(-0.5)), hnodes, inodes) # excellent idea from the oringinal author
    who = rand(Normal(0, onodes^(-0.5)), onodes, hnodes)
    return NetInfo(inodes, hnodes, onodes, wih, who, lr)
end

InitNet (generic function with 1 method)

Forward-feed,

In [41]:
function Query(net::NetInfo, inputs)
    # signal,input to the hidden
    hidden_inputs = net.wih * inputs
    hidden_outputs = sigmoid.(hidden_inputs)
    # signal,the hidden,to output
    final_inputs = net.who * hidden_outputs
    final_outputs = sigmoid.(final_inputs)
    return  final_outputs
end

Query (generic function with 1 method)

Backfoward feed

In [62]:
function Train!(net::NetInfo, inputs, targets)
    # part 1,forward feed
    ## signal,input to the hidden
    hidden_inputs = net.wih * inputs
    hidden_outputs = sigmoid.(hidden_inputs)
    ## signal,the hidden,to output
    final_inputs = net.who * hidden_outputs
    final_outputs = sigmoid.(final_inputs)
    # part 2：back forward
    ## output error=target - actual
    output_errors = targets - final_outputs # deviation
    hidden_errors = net.who' * output_errors
    net.who += net.lr .* (output_errors .* final_outputs .* (1.0 .- final_outputs)) * hidden_outputs'
    net.wih += net.lr .* (hidden_errors .* hidden_outputs .* (1.0 .- hidden_outputs)) * inputs'
end

Train! (generic function with 1 method)

### 1.4.2  Practicing on a smple dataset

In [59]:
using DelimitedFiles
training_data_file = readdlm("D:/DS_thing/000/mnist_dataset/mnist_train_100.csv", ',');  # a sample

arguments setting

In [69]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10
learning_rate = 0.3;

experiment in the sample dataset

In [72]:
train_data_size = 100
test_data_size = 10;

normalization and reshape

In [61]:
record=1
targets = zeros(output_nodes) .+ 0.01
targets[round(Int, training_data_file[record, 1]) + 1] = 0.99
targets = reshape(targets, 10, 1)

10×1 Array{Float64,2}:
 0.01
 0.01
 0.01
 0.01
 0.01
 0.99
 0.01
 0.01
 0.01
 0.01

In [70]:
net_test = InitNet(input_nodes,hidden_nodes,output_nodes,learning_rate)

NetInfo(784, 100, 10, [-0.0121131 -0.16516 … 0.0222304 -0.0884992; 0.0105856 0.0614349 … 0.169588 0.10256; … ; 0.0043748 -0.0316893 … -0.159509 0.0769326; 0.0515735 0.177797 … 0.0215483 0.00844891], [0.218044 -0.178091 … -0.266258 0.17956; -0.110988 -0.198228 … -0.107388 0.0746235; … ; 0.583456 -0.293129 … -0.0140891 -0.217309; 0.205378 0.198045 … 0.583077 -0.264263], 0.3)

In [71]:
#net_test=InitNet
for record = 1:train_data_size
    inputs = (training_data_file[record, 2:end] ./ 255.0 .* 0.99) .+ 0.01
    inputs = reshape(inputs, input_nodes, 1) # 
    targets = zeros(output_nodes) .+ 0.01
    targets[Int(training_data_file[record, 1]) + 1] = 0.99
    targets = reshape(targets, 10, 1)

    Train!(net_test, inputs, targets)
end

adding the test set sample

In [ ]:
test_data_file = readdlm("D:/DS_thing/000/mnist_dataset/mnist_test_10.csv", ',')

In [77]:
scorecard = []

for record = 1:test_data_size
    correct_label = Int(test_data_file[record, 1])
    inputs = (test_data_file[record, 2:end] ./ 255.0 .* 0.99) .+ 0.01
    inputs = reshape(inputs, input_nodes, 1) # adjust the dim
    
    outputs = Query(net_test, inputs)
      # 1value，2 index
    label = findmax(outputs)[2][1]
    if (label - 1 == correct_label)
        append!(scorecard, 1)
    else
        append!(scorecard, 0)
    end

end

In [78]:
print("performance = ", sum(scorecard) / length(scorecard))

performance = 0.6

### 1.4.3  Practicing on the whole datset 

In [79]:
test_data_file = readdlm("D:/DS_thing/000/mnist_dataset/mnist_test.csv", ',')
training_data_file = readdlm("D:/DS_thing/000/mnist_dataset/mnist_train.csv", ',');

In [80]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10
learning_rate = 0.3
train_data_size = 60000
test_data_size = 10000
net_test = InitNet(input_nodes,hidden_nodes,output_nodes,learning_rate)

NetInfo(784, 100, 10, [-0.00852818 -0.064523 … -0.0254088 -0.0349585; 0.0655188 0.142846 … -0.0506445 -0.0946943; … ; -0.083134 -0.110499 … -0.106505 -0.0573991; -0.131888 -0.0403696 … -0.037773 -0.103555], [-0.0714624 -0.7626 … 0.179258 -0.495606; -0.036007 0.137162 … -0.265109 0.340104; … ; 0.387522 0.0661092 … 0.480359 -0.494109; -0.746022 -0.178318 … 0.447564 -0.168922], 0.3)

In [84]:
for record = 1:train_data_size
    inputs = (training_data_file[record, 2:end] ./ 255.0 .* 0.99) .+ 0.01
    inputs = reshape(inputs, input_nodes, 1) # 
    targets = zeros(output_nodes) .+ 0.01
    targets[Int(training_data_file[record, 1]) + 1] = 0.99
    targets = reshape(targets, 10, 1)

    Train!(net_test, inputs, targets)
end

In [89]:
training_data_file[1,2:3]

2-element Array{Float64,1}:
 0.0
 0.0

In [83]:
scorecard = []

for record = 1:test_data_size
   # print(record，":----")
    correct_label = Int(test_data_file[record, 1])
    inputs = (test_data_file[record, 2:end] ./ 255.0 .* 0.99) .+ 0.01
    inputs = reshape(inputs, input_nodes, 1) # adjust the dim
    
    outputs = Query(net_test, inputs)
      # 1value，2 index
    label = findmax(outputs)[2][1]
    if (label - 1 == correct_label)
        append!(scorecard, 1)
    else
        append!(scorecard, 0)
    end

end

print("performance = ", sum(scorecard) / length(scorecard))

performance = 0.0969

epochs

In [98]:
epochs =5

for e=1:epochs
    for record = 1:train_data_size
        inputs = (training_data_file[record, 2:end] ./ 255.0 .* 0.99) .+ 0.01
        inputs = reshape(inputs, input_nodes, 1) # 
        targets = zeros(output_nodes) .+ 0.01
        targets[Int(training_data_file[record, 1]) + 1] = 0.99
        targets = reshape(targets, 10, 1)

        Train!(net_test, inputs, targets)
    end
end